<a href="https://colab.research.google.com/github/tavochopadron-cyber/MUCHACHAM/blob/main/MUCHACHAM/muchacham.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# app.py
import os
import tempfile
import json
import base64
import secrets
from datetime import datetime
from collections import defaultdict

import numpy as np
import joblib
import matplotlib.pyplot as plt
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

import gradio as gr

# ---------------- CONFIG ----------------
MONGO_URI = os.environ.get("MONGO_URI", None)
if MONGO_URI:
    client = MongoClient(MONGO_URI)
    db = client.get_database("bricos_db")
    coleccion = db.get_collection("candidatos")
else:
    client = None
    coleccion = None

PUESTOS = ["CARGADOR", "VENTAS/COBRANZA", "REPARTIDOR", "LIMPIEZA", "ADMINISTRATIVO"]
NUM_PREGUNTAS = 15

PREGUNTAS = {
    "CARGADOR": [
        "¿Te consideras una persona que trabaja bien bajo presión?",
        "¿Mantienes la calma cuando hay mucha carga pendiente?",
        "¿Te comunicas con tu equipo antes de mover mercancía pesada?",
        "¿Sueles revisar la mercancía antes de moverla?",
        "¿Sigues las indicaciones de seguridad al levantar objetos?",
        "¿Sueles organizar tu área de trabajo para evitar accidentes?",
        "¿Te adaptas fácilmente cuando cambian las prioridades del día?",
        "¿Puedes trabajar en equipo sin conflictos?",
        "¿Detectas a tiempo mercancía dañada o mal etiquetada?",
        "¿Tienes experiencia usando patín hidráulico?",
        "¿Tomas descansos solo cuando están permitidos?",
        "¿Te consideras responsable con los tiempos asignados?",
        "¿Puedes mantener una actitud positiva en días complicados?",
        "¿Avisas de inmediato cuando encuentras alguna irregularidad?",
        "¿Sabes seguir instrucciones sin supervisión constante?"
    ],
    "VENTAS/COBRANZA": [
        "¿Te sientes cómodo hablando con clientes diariamente?",
        "¿Puedes mantener la calma cuando un cliente está molesto?",
        "¿Sueles ser persistente sin caer en insistencia excesiva?",
        "¿Llevas un control ordenado de tus cuentas o visitas?",
        "¿Te adaptas bien a trabajar con metas?",
        "¿Eres capaz de negociar sin generar conflicto?",
        "¿Analizas si un cliente representa riesgo de pago?",
        "¿Puedes comunicarte con claridad por teléfono?",
        "¿Te consideras una persona paciente?",
        "¿Has usado sistemas de registro o facturación?",
        "¿Sueles detectar las necesidades del cliente rápidamente?",
        "¿Puedes aceptar un “no” sin frustrarte?",
        "¿Informas a tiempo cuando detectas un atraso en un cliente?",
        "¿Te mantienes organizado aun con muchas cuentas simultáneas?",
        "¿Te sientes capaz de recuperar pagos atrasados?"
    ],
    "REPARTIDOR": [
        "¿Te orientas bien usando GPS o mapas digitales?",
        "¿Te consideras una persona puntual?",
        "¿Puedes manejar bien el estrés cuando las entregas aumentan?",
        "¿Verificas la mercancía antes de salir a ruta?",
        "¿Mantienes una actitud amable en todas tus entregas?",
        "¿Reportas de inmediato cualquier contratiempo en ruta?",
        "¿Sigues rutas nuevas sin problema?",
        "¿Puedes manejar distancias largas sin perder concentración?",
        "¿Has manejado antes un vehículo de reparto?",
        "¿Cuidas la mercancía para que llegue en buen estado?",
        "¿Te adaptas rápido cuando hay cambios de último momento?",
        "¿Puedes tratar con clientes molestos sin perder la calma?",
        "¿Administra bien el dinero o comprobantes durante las entregas?",
        "¿Conoces normas básicas de tránsito?",
        "¿Tomas medidas para asegurar el vehículo durante la ruta?"
    ],
    "LIMPIEZA": [
        "¿Te consideras una persona organizada?",
        "¿Puedes trabajar sin supervisión directa?",
        "¿Sueles mantener una actitud tranquila cuando te piden limpiar algo urgente?",
        "¿Puedes seguir protocolos de limpieza establecidos?",
        "¿Sabes usar productos básicos de limpieza?",
        "¿Informas cuando encuentras algo fuera de lugar?",
        "¿Puedes trabajar en áreas donde hay otras personas sin interrumpirlas?",
        "¿Te adaptas bien a tareas repetitivas?",
        "¿Cuidas el uso de químicos o materiales?",
        "¿Priorizas las tareas cuando hay muchas pendientes?",
        "¿Sigues medidas de seguridad al limpiar áreas riesgosas?",
        "¿Aceptas retroalimentación sin problema?",
        "¿Puedes levantar objetos ligeros o mover mobiliario pequeño?",
        "¿Mantienes discreción cuando encuentras información u objetos sensibles?",
        "¿Te consideras una persona puntual y constante?"
    ],
    "ADMINISTRATIVO": [
        "¿Te consideras una persona organizada?",
        "¿Puedes trabajar con varias tareas al mismo tiempo?",
        "¿Tienes experiencia usando computadoras o software de oficina?",
        "¿Te molesta trabajar bajo presión?",
        "¿Verificas tu trabajo antes de entregarlo para evitar errores?",
        "¿Comunicas a tiempo cuando te falta información para avanzar?",
        "¿Puedes mantener la calma con jefes o usuarios molestos?",
        "¿Te adaptas fácilmente cuando cambian prioridades?",
        "¿Puedes manejar información confidencial con responsabilidad?",
        "¿Sigues instrucciones con precisión?",
        "¿Te sientes cómodo haciendo capturas o registros repetitivos?",
        "¿Sueles detectar errores en documentos antes de enviarlos?",
        "¿Te consideras una persona puntual?",
        "¿Te llevas bien con diferentes áreas de trabajo?",
        "¿Puedes trabajar sin supervisión directa?"
    ]
}

MODEL_FILE = "modelo_muchacham.pkl"

# ---------------- DATOS SINTÉTICOS Y ENTRENAMIENTO ----------------
def generar_datos_sinteticos(n_por_clase=300, seed=42):
    np.random.seed(seed)
    X = []
    y = []
    m = len(PUESTOS)
    for idx, puesto in enumerate(PUESTOS):
        for _ in range(n_por_clase):
            vect = np.zeros(NUM_PREGUNTAS * m, dtype=int)
            for j, p in enumerate(PUESTOS):
                start = j * NUM_PREGUNTAS
                if p == puesto:
                    probs = np.full(NUM_PREGUNTAS, 0.78)
                else:
                    probs = np.full(NUM_PREGUNTAS, 0.22)
                sampled = (np.random.rand(NUM_PREGUNTAS) < probs).astype(int)
                vect[start:start + NUM_PREGUNTAS] = sampled
            X.append(vect)
            y.append(idx)
    return np.array(X), np.array(y)

def entrenar_o_cargar_modelo(path=MODEL_FILE):
    if os.path.exists(path):
        modelo, scaler = joblib.load(path)
        return modelo, scaler
    X, y = generar_datos_sinteticos()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    modelo = MLPClassifier(hidden_layer_sizes=(60, 30), activation="relu", max_iter=500, random_state=1)
    modelo.fit(X_train_s, y_train)
    joblib.dump((modelo, scaler), path)
    return modelo, scaler

modelo, scaler = entrenar_o_cargar_modelo()

# ---------------- UTILIDADES ----------------
def respuestas_a_vector(respuestas_por_puesto):
    vect = []
    for puesto in PUESTOS:
        vals = respuestas_por_puesto.get(puesto, ["N"] * NUM_PREGUNTAS)
        if len(vals) != NUM_PREGUNTAS:
            vals = ["N"] * NUM_PREGUNTAS
        vect.extend([1 if str(x).strip().upper().startswith("S") else 0 for x in vals])
    return np.array(vect, dtype=int).reshape(1, -1)

def generar_pdf_minimalista(registro):
    tf = tempfile.NamedTemporaryFile(delete=False, suffix=".pdf")
    c = canvas.Canvas(tf.name, pagesize=letter)
    c.setFont("Helvetica-Bold", 14)
    y = 750
    c.drawString(50, y, "Reporte MUCHACHAM (Minimalista)")
    c.setFont("Helvetica", 11)
    y -= 30
    c.drawString(50, y, f"Nombre: {registro.get('nombre','')}")
    y -= 18
    c.drawString(50, y, f"Edad: {registro.get('edad','')}")
    y -= 18
    c.drawString(50, y, f"Sexo: {registro.get('sexo','')}")
    y -= 18
    c.drawString(50, y, f"Puesto recomendado: {registro.get('puesto_predicho','')}")
    y -= 18
    c.drawString(50, y, f"Probabilidad: {registro.get('probabilidad_puesto_predicho',0)*100:.2f}%")
    y -= 28
    c.setFont("Helvetica-Bold", 12)
    c.drawString(50, y, "Probabilidades por puesto:")
    c.setFont("Helvetica", 10)
    y -= 20
    probs = registro.get("probabilidades", {})
    for p in PUESTOS:
        val = probs.get(p, 0)
        c.drawString(60, y, f"- {p}: {val*100:.2f}%")
        y -= 14
        if y < 80:
            c.showPage()
            y = 750
    y -= 10
    c.setFont("Helvetica-Oblique", 9)
    c.drawString(50, y, f"Fecha: {datetime.utcnow().isoformat()} UTC")
    c.save()
    return tf.name

def generar_grafica_probabilidades(probs_dict):
    labels = list(probs_dict.keys())
    values = [probs_dict[k] for k in labels]
    fig, ax = plt.subplots(figsize=(6, 3.5))
    ax.bar(labels, np.array(values) * 100)
    ax.set_ylabel("Probabilidad (%)")
    ax.set_ylim(0, 100)
    ax.set_title("Probabilidades por puesto")
    plt.xticks(rotation=30, ha="right")
    tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".png")
    fig.tight_layout()
    fig.savefig(tmp.name)
    plt.close(fig)
    return tmp.name

# ---------------- LÓGICA PRINCIPAL ----------------
def predecir_y_guardar(nombre, edad, sexo, correo, telefono, *respuestas_flat):
    # reconstruir dict por puesto
    respuestas_por_puesto = {}
    idx = 0
    for puesto in PUESTOS:
        block = []
        for _ in range(NUM_PREGUNTAS):
            if idx < len(respuestas_flat):
                block.append(respuestas_flat[idx])
            else:
                block.append("N")
            idx += 1
        respuestas_por_puesto[puesto] = block

    vec = respuestas_a_vector(respuestas_por_puesto)
    vec_s = scaler.transform(vec)
    probs = modelo.predict_proba(vec_s)[0]
    idx_max = int(np.argmax(probs))
    puesto_pred = PUESTOS[idx_max]
    prob_puesto = float(probs[idx_max])
    probs_dict = {PUESTOS[i]: float(probs[i]) for i in range(len(PUESTOS))}

    registro = {
        "nombre": nombre,
        "edad": edad,
        "sexo": sexo,
        "correo_cifrado": base64.urlsafe_b64encode(correo.encode("utf-8")).decode("utf-8"),
        "telefono_cifrado": base64.urlsafe_b64encode(telefono.encode("utf-8")).decode("utf-8"),
        "respuestas_por_puesto": respuestas_por_puesto,
        "puesto_predicho": puesto_pred,
        "probabilidad_puesto_predicho": prob_puesto,
        "probabilidades": probs_dict,
        "fecha": datetime.utcnow()
    }

    insert_result = None
    if coleccion:
        try:
            insert_result = str(coleccion.insert_one(registro).inserted_id)
        except Exception as e:
            insert_result = f"ERROR: {str(e)}"

    # generar pdf y grafica
    pdf_path = generar_pdf_minimalista(registro)
    graf_path = generar_grafica_probabilidades(probs_dict)

    salida = {
        "registro": registro,
        "mongo_id": insert_result,
        "pdf_path": os.path.basename(pdf_path),
        "graf_path": os.path.basename(graf_path)
    }
    # leer pdf y graf como binario base64 para descarga en Gradio
    with open(pdf_path, "rb") as f:
        pdf_b64 = base64.b64encode(f.read()).decode("utf-8")
    with open(graf_path, "rb") as f:
        graf_b64 = base64.b64encode(f.read()).decode("utf-8")
    salida["pdf_b64"] = pdf_b64
    salida["graf_b64"] = graf_b64

    return salida

# ---------------- INTERFAZ GRADIO ----------------
def construir_interfaz():
    with gr.Blocks() as demo:
        gr.Markdown("## MUCHACHAM — Evaluación automática con Red Neuronal")
        with gr.Row():
            with gr.Column(scale=1):
                nombre = gr.Textbox(label="Nombre completo", lines=1)
                edad = gr.Textbox(label="Edad", lines=1)
                sexo = gr.Dropdown(label="Sexo", choices=["M", "F", "Otro"], value="M")
                correo = gr.Textbox(label="Correo electrónico", lines=1)
                telefono = gr.Textbox(label="Teléfono", lines=1)
            with gr.Column(scale=1):
                gr.Markdown("Rellena las tarjetas con S (Sí) o N (No). Cuando termines, pulsa **Generar PDF y Guardar**.")
        # preguntas en accordions
        inputs_flat = []
        for puesto in PUESTOS:
            with gr.Accordion(puesto, open=False):
                for i, texto in enumerate(PREGUNTAS[puesto]):
                    r = gr.Radio(choices=["S", "N"], value="N", label=f"{i+1}. {texto}")
                    inputs_flat.append(r)
        btn = gr.Button("Generar PDF y Guardar")
        salida_json = gr.JSON(label="Resultado (Registro + Metadatos)")
        descarga_pdf = gr.File(label="Descargar PDF")
        graf_img = gr.Image(label="Gráfica de probabilidades")

        def ejecutar_interfaz(nombre_, edad_, sexo_, correo_, telefono_, *args):
            res = predecir_y_guardar(nombre_, edad_, sexo_, correo_, telefono_, *args)
            # crear archivos temporales para que Gradio pueda servirlos
            pdf_tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".pdf")
            pdf_tmp.write(base64.b64decode(res["pdf_b64"]))
            pdf_tmp.flush()
            graf_tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".png")
            graf_tmp.write(base64.b64decode(res["graf_b64"]))
            graf_tmp.flush()
            return res, pdf_tmp.name, graf_tmp.name

        btn.click(fn=ejecutar_interfaz, inputs=[nombre, edad, sexo, correo, telefono] + inputs_flat,
                  outputs=[salida_json, descarga_pdf, graf_img])

        # Dashboard: mostrar gráficas agregadas si hay Mongo
        with gr.Accordion("Dashboard (estadísticas)", open=False):
            stats_btn = gr.Button("Actualizar estadísticas")
            stats_img = gr.Image(label="Candidatos por puesto")
            stats_txt = gr.Markdown()

            def generar_dashboard():
                if not coleccion:
                    return None, "Mongo no configurado. Define MONGO_URI como secret."
                pipeline = [
                    {"$group": {"_id": "$puesto_predicho", "count": {"$sum": 1}}}
                ]
                agg = list(coleccion.aggregate(pipeline))
                counts = {item["_id"]: item["count"] for item in agg}
                # asegurar todos los puestos
                counts = {p: counts.get(p, 0) for p in PUESTOS}
                # graficar
                fig, ax = plt.subplots(figsize=(6,3.5))
                ax.bar(list(counts.keys()), list(counts.values()))
                ax.set_title("Candidatos guardados por puesto")
                ax.set_ylabel("Cantidad")
                plt.xticks(rotation=30, ha="right")
                tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".png")
                fig.tight_layout()
                fig.savefig(tmp.name)
                plt.close(fig)
                md = "#### Totales\n\n"
                total = sum(counts.values())
                md += f"- Total candidatos: **{total}**\n"
                for k,v in counts.items():
                    md += f"- {k}: **{v}**\n"
                return tmp.name, md

            stats_btn.click(fn=generar_dashboard, inputs=[], outputs=[stats_img, stats_txt])

    return demo

if __name__ == "__main__":
    demo = construir_interfaz()
    demo.launch()
